# Vacation-planning weather analysis

In [1]:
# Import all dependencies and styles needed for notebook.

# Set Jupyter Notebook to display plots inline.
%matplotlib inline

# Import dependencies.
from matplotlib import style
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime as dt
from datetime import datetime, timedelta
from matplotlib.dates import MonthLocator, DateFormatter

# Import sqlalchemy dependencies.
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

# Use fancy matplotlib style for plots.
style.use('fivethirtyeight')

# Use Sqlalchemy library (ORM) to read sqlite database into Python object

In [2]:
# Create db engine for sqlite db.
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

# Create inspector object to view tables and columns in sqlite db.
inspector = inspect(engine)

# Use inspector object to print all table names and column schemas.
tables = inspector.get_table_names()
for t in tables:
    # Print table name.
    print(t)
    # Get all schema info for each column.
    column_info = inspector.get_columns(t)
    # For each column, print out schema info.
    for x in column_info:
        print(x)

measurement
{'name': 'id', 'type': INTEGER(), 'nullable': False, 'default': None, 'autoincrement': 'auto', 'primary_key': 1}
{'name': 'station', 'type': TEXT(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'date', 'type': TEXT(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'prcp', 'type': FLOAT(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'tobs', 'type': FLOAT(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
station
{'name': 'id', 'type': INTEGER(), 'nullable': False, 'default': None, 'autoincrement': 'auto', 'primary_key': 1}
{'name': 'station', 'type': TEXT(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'name', 'type': TEXT(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'latitude', 'type': FLOAT(), 'nullable': True, 'default': None, 'autoin

In [3]:
# Start query session for db engine.
session = Session(engine)

# Read (reflect) sqlite db into SQLAlchemy Base object.
Base = automap_base()
Base.prepare(engine, reflect = True)

# Create reference to each table in Base.
Station = Base.classes.station
Measurement = Base.classes.measurement

# Exploratory Climate Analysis

In [4]:
# View first 5 records from prcp column
result = session.query(Measurement.prcp).limit(5).all()
result

[(0.08), (0.0), (0.0), (0.0), (None)]

In [5]:
# View first 5 records from date column.
result = session.query(Measurement.date).limit(5).all()
result

[('2010-01-01'),
 ('2010-01-02'),
 ('2010-01-03'),
 ('2010-01-04'),
 ('2010-01-06')]

In [10]:
# Compute date one year before last date present in database.

# Query last date in database.
last_date = session.query(Measurement.date).order_by(Measurement.date.desc()).first()

# Print last_date
last_date

# ('2017-08-23',) looks like a tuple of strings...

# Index into last_date.
last_date_string = last_date[0]

# Format string as datetime object.
format_str = '%Y-%m-%d'
datetime_obj = dt.datetime.strptime(last_date_string, format_str)

# Subtract 365 years from datetime object.
year_ago = datetime_obj - dt.timedelta(days = 365)

In [11]:
# Get date and precipitation over last year of data.
results = session.query(Measurement.date, Measurement.prcp).filter(Measurement.date >= year_ago).all()

# Save query results as Pandas DataFrame
precip_df = pd.DataFrame(results)

# Set df index to date column
precip_df.set_index('date')

# Sort df by date
precip_df.sort_index()

precip_df.head()

,date,prcp
0,2016-08-24,0.08
1,2016-08-25,0.08
2,2016-08-26,0.00
3,2016-08-27,0.00
4,2016-08-28,0.01


In [ ]:
# Use Pandas Plotting with Matplotlib to plot the data
plt.bar(precip_df["date"], precip_df["prcp"])

fig, ax = plt.subplots()
ax.plot('date', 'adj_close', data=data)

ax.xaxis.set_major_locator(years)
ax.xaxis.set_major_formatter(years_fmt)
ax.xaxis.set_minor_locator(months)

# Use Zach's slacked webpage.

plt.xlabel('Date')
plt.ylabel('Precipitation (inches)')
plt.title('Precipitation')
plt.show()

In [ ]:
# Precipitation summary statistics
precip_df.describe()

In [ ]:
# Design a query to show how many stations are available in this dataset?


In [ ]:
# What are the most active stations? (i.e. what stations have the most rows)?
# List the stations and the counts in descending order.


In [ ]:
# Using the station id from the previous query, calculate the lowest temperature recorded, 
# highest temperature recorded, and average temperature of the most active station?


In [ ]:
# Choose the station with the highest number of temperature observations.
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram


## Bonus Challenge Assignment

In [ ]:
# This function called `calc_temps` will accept start date and end date in the format '%Y-%m-%d' 
# and return the minimum, average, and maximum temperatures for that range of dates
def calc_temps(start_date, end_date):
    """TMIN, TAVG, and TMAX for a list of dates.
    
    Args:
        start_date (string): A date string in the format %Y-%m-%d
        end_date (string): A date string in the format %Y-%m-%d
        
    Returns:
        TMIN, TAVE, and TMAX
    """
    
    return session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
        filter(Measurement.date >= start_date).filter(Measurement.date <= end_date).all()

# function usage example
print(calc_temps('2012-02-28', '2012-03-05'))

In [ ]:
# Use your previous function `calc_temps` to calculate the tmin, tavg, and tmax 
# for your trip using the previous year's data for those same dates.


In [ ]:
# Plot the results from your previous query as a bar chart. 
# Use "Trip Avg Temp" as your Title
# Use the average temperature for the y value
# Use the peak-to-peak (tmax-tmin) value as the y error bar (yerr)


In [ ]:
# Calculate the total amount of rainfall per weather station for your trip dates using the previous year's matching dates.
# Sort this in descending order by precipitation amount and list the station, name, latitude, longitude, and elevation



In [ ]:
# Create a query that will calculate the daily normals 
# (i.e. the averages for tmin, tmax, and tavg for all historic data matching a specific month and day)

def daily_normals(date):
    """Daily Normals.
    
    Args:
        date (str): A date string in the format '%m-%d'
        
    Returns:
        A list of tuples containing the daily normals, tmin, tavg, and tmax
    
    """
    
    sel = [func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)]
    return session.query(*sel).filter(func.strftime("%m-%d", Measurement.date) == date).all()
    
daily_normals("01-01")

In [ ]:
# calculate the daily normals for your trip
# push each tuple of calculations into a list called `normals`

# Set the start and end date of the trip

# Use the start and end date to create a range of dates

# Stip off the year and save a list of %m-%d strings

# Loop through the list of %m-%d strings and calculate the normals for each date


In [ ]:
# Load the previous query results into a Pandas DataFrame and add the `trip_dates` range as the `date` index


In [ ]:
# Plot the daily normals as an area plot with `stacked=False`
